# Excerise 3

In [77]:
import numpy as np
import pandas as pd
import os
import ast
import git
from git import RemoteProgress
from git import Repo
from datetime import datetime, date
import matplotlib.pyplot as plt
import seaborn as sns
from time import strptime
%matplotlib inline

In [78]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

# Git Repo

In [79]:
#Spring Framework File 1
#remote_link = "https://github.com/spring-projects/spring-framework"
#local_link = "C:\\Users\\User\\repo\\spring-framework"
#repo = Repo(local_link)
#fixing_commit = "e2d6e709c3c65a4951eb096843ee75d5200cfcad"
#affected_files = ["spring-webmvc/src/main/java/org/springframework/web/servlet/ResourceServlet.java"]

#Spring Framework File 2
#remote_link = "https://github.com/spring-projects/spring-framework"
#local_link = "C:\\Users\\User\\repo\\spring-framework"
#repo = Repo(local_link)
#fixing_commit = "e2d6e709c3c65a4951eb096843ee75d5200cfcad"
#affected_files = ["spring-webmvc/src/test/java/org/springframework/web/servlet/ResourceServletTests.java"]


# Camel
#remote_link = "https://github.com/apache/camel"
#local_link = "C:\\Users\\User\\repo\\camel"
#repo = Repo(local_link)
#fixing_commit = "c93a87c36aa4d14ad6f7ee1df9507fa2ca1fd91"
#affected_files = ["camel-core/src/main/java/org/apache/camel/model/dataformat/JsonDataFormat.java"]


# Tika
remote_link = "https://github.com/apache/tika"
local_link = "C:\\Users\\User\\repo\\tika"
repo = Repo(local_link)
fixing_commit = "62926cae31a02d4f23d21148435804b96c543cc"
affected_files = ["tika-parsers/src/main/java/org/apache/tika/parser/iptc/IptcAnpaParser.java"]


# Function findStartEnd

In [80]:
def findStartEnd(file, line_affected):
    start = -1
    end = -1
    current_line = 1
    for line in file:
        if current_line == line_affected:
            i = current_line
            found = False
            while not found:
                i = i - 1
                if i == -1:
                    break
                if "{" in file[i - 1]:
                    found = True
                    start = i
                    break;
            found = False
            i = current_line
            while not found:
                i = i + 1
                if i >= (len(file) - 1):
                    break
                if "}" in file[i - 1]:
                    found = True
                    end = i
                    break   
        current_line = current_line + 1
    return start, end   

# Function makeList

In [81]:
def makeList(input_list, value):
    found = False
    for item in input_list:
        if item["commit"] == value:
            item["count"] = item["count"] + 1
            found = True
    if not found:
        input_list.append({"commit": value, "count": 1})
    return input_list

# Function findAddDelete

In [82]:
def findAddDelete(line):
    if line[0] == "@@":
        delete = line[1]
        delete = delete[1:]
        delete = delete.split(",")
        if len(delete) < 2:
            delete.append("1")
        
        add = line[2]
        add = add[1:]
        add = add.split(",")
        if len(add) < 2:
            add.append("1")
                
        return add, delete

# Function findCommon

In [83]:
def findCommon(commits_found, exclude):
    maximum = -1
    commit_found = None
    for commit in commits_found:
        if exclude in commit["commit"]:
            continue
        if commit["count"] > maximum:
            commit_found = commit["commit"]
            maximum = commit["count"]
            
    return commit_found, maximum

# Function getVCC

In [84]:
def getVCC(files, parameters):
    for file in files:
        diff_of_file = repo.git.diff(fixing_commit + "^" + ".." + fixing_commit, "-U0", "--", file).splitlines()
        diff_of_file = diff_of_file[4:]
        raw_commit_file = repo.git.show(fixing_commit + ":" + file).splitlines()
        commits_found = []
        i = 0
        while i < len(diff_of_file):
            split_line = diff_of_file[i].split()
            if split_line[0] == "@@":
                additions, deletions = findAddDelete(split_line)
                if deletions[1] != 0:
                    j = 0
                    line_number = int(deletions[0])
                    while j < int(deletions[1]):
                        j = j + 1
                        affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(line_number) + "," + str(line_number), fixing_commit, "--", file).splitlines()
                        split_line = affected_file_diff[0].split()
                        makeList(commits_found, split_line[0])
                    i = i + j
                if additions[1] != 0:
                    j = 0
                    line_number = int(additions[0])
                    while j < int(additions[1]):
                        j = j + 1
                        start, end = findStartEnd(raw_commit_file, j + line_number)                         
                        if start != -1 and end != -1:
                            affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(start) + "," + str(end), fixing_commit, "--", file).splitlines()                         
                        else:
                            affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(line_number) + "," + str(line_number), fixing_commit, "--", file).splitlines()             
                        for line in affected_file_diff:
                                split_line = line.split()
                                makeList(commits_found, split_line[0])
                        line_number = line_number + 1
                    i = i + j                
            i = i + 1
    return findCommon(commits_found, fixing_commit)

In [85]:
parameters = None
# parameters = "-w"
# parameters = "-wM"
# parameters = "-wC"
# parameters = "-wCC"
# parameters = "-wCCC"

VCC = getVCC(affected_files, parameters)

print("VCC Commit:", VCC[0])
print("Frequency:", VCC[1])

VCC Commit: de7092125649cd61840e27b1f70d68cfab910d89
Frequency: 29
